In [350]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import warnings
warnings.filterwarnings(action='ignore')
from catboost import *
import sklearn
import math
from sklearn.preprocessing import LabelEncoder
import math

def log(train,test):
    col_list = train.columns
    for col in col_list :
        if '제곱' in col:
            train[col+'log'] = np.log1p(train[col])
            test[col+'log'] = np.log1p(test[col])
    return train,test

def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    '''모든값이 결측값이면 제거'''
    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test
   
def make_train_test_dataset(train,test):
    train_x = train.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w

def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

def multi_col(train,test) : 
    col_list = train.columns
    for col in col_list :
        if col == 'LINE':
            continue
        train[col+'제곱'] = (train[col]**2)//100
        test[col+'제곱'] = (test[col]**2)//100
    return train,test

# predict 0

In [351]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)


# 강한 추세성보이는 칼럼 중 일부 제거
'''
drop_a = ['X_1167','X_1168','X_1169','X_1170','X_1171','X_1274','X_1275','X_1276','X_1277','X_1278']
drop_t = ['X_529','X_530','X_531','X_532','X_533','X_600','X_601','X_602','X_603','X_604']

trainA_31_x = trainA_31_x.drop(columns=drop_a)
testA_31_x = testA_31_x.drop(columns=drop_a)
train_T_31_x = train_T_31_x.drop(columns=drop_t)
test_T_31_x = test_T_31_x.drop(columns=drop_t)
'''

# {'onehot', 'onehot-dense', 'ordinal'}
from sklearn.preprocessing import KBinsDiscretizer
enc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy="kmeans", random_state=1414)

featurest = train_T_31_x.columns[1:]
enc.fit(train_T_31_x[featurest])
binned_df_train = enc.transform(train_T_31_x[featurest])
binned_df_test = enc.transform(test_T_31_x[featurest])
for i, featurest in enumerate(featurest):
    train_T_31_x[f"{featurest}_binned"] = binned_df_train[:,i]
    test_T_31_x[f"{featurest}_binned"] = binned_df_test[:,i]

enc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy="kmeans", random_state=1414)
features = trainA_31_x.columns[1:]
enc.fit(trainA_31_x[features])
binned_df_train = enc.transform(trainA_31_x[features])
binned_df_test = enc.transform(testA_31_x[features])
for i, feature in enumerate(features):
    trainA_31_x[f"{feature}_binned"] = binned_df_train[:,i]
    testA_31_x[f"{feature}_binned"] = binned_df_test[:,i]

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 4233) (67, 4233) (343, 1355) (239, 1355)


In [352]:
# binary
trainA_31_w.loc[(trainA_31_w['Y_Class']==2),'Y_Class']=1
train_T_31_w.loc[(train_T_31_w['Y_Class']==2),'Y_Class']=1
traint = pd.concat([train_T_31_x,train_T_31_w],axis = 1)
traina = pd.concat([trainA_31_x,trainA_31_w],axis = 1)

In [353]:
corr_df = traint.corr()
corr_df = corr_df.apply(lambda x: round(x ,5))
s = corr_df.unstack()
성장 = pd.DataFrame(s['Y_Class']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
under_05per_t = 성장[성장[0]<0.002].index
성장 = 성장.sort_values(by=[0],ascending=False).reset_index()
droppc_t = []
for col in traint.columns[1:678] :
    col_corr = traint[[col,col+'_binned']].corr()[col+'_binned'][0]
    if col_corr > 0.70 :        
        origin = 성장[성장['index']==col][0].values[0]
        binned = 성장[성장['index']==col+'_binned'][0].values[0]
        if origin >= binned :
            droppc_t.append(col+'_binned')
        else :
            droppc_t.append(col)
dropt = set(list(under_05per_t)+droppc_t+['LINE'])
print(len(under_05per_t), len(droppc_t))

134 360


In [354]:
corr_df = traina.corr()
corr_df = corr_df.apply(lambda x: round(x ,5))
s = corr_df.unstack()
성장 = pd.DataFrame(s['Y_Class']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
under_05per_a = 성장[성장[0]<0.007].index
성장 = 성장.sort_values(by=[0],ascending=False).reset_index()
droppc_a = []
for col in traina.columns[1:2117] :
    col_corr = traina[[col,col+'_binned']].corr()[col+'_binned'][0]
    if col_corr > 0.70 :    
        origin = 성장[성장['index']==col][0].values[0]
        binned = 성장[성장['index']==col+'_binned'][0].values[0]
        if origin >= binned :
            droppc_a.append(col+'_binned')
        else :
            droppc_a.append(col)
dropa = set(list(under_05per_a)+droppc_a+['LINE'])
print(len(under_05per_a), len(droppc_a))

95 1955


In [355]:
trainA_31_x = trainA_31_x.drop(columns=dropa)
testA_31_x = testA_31_x.drop(columns=dropa)
train_T_31_x = train_T_31_x.drop(columns=dropt)
test_T_31_x = test_T_31_x.drop(columns=dropt)

In [356]:
print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 2220) (67, 2220) (343, 866) (239, 866)


In [357]:
catreg1 = CatBoostRegressor(random_state=1234,
                           depth=6,
                           verbose=500,
                           iterations=9000,
                           learning_rate=0.006)

catreg2 = CatBoostRegressor(random_state=1234,
                           depth=5,
                           verbose=500,
                           iterations=5500,
                           learning_rate=0.018)

In [358]:
catreg1.fit(trainA_31_x, trainA_31_y)
pred_a = catreg1.predict(testA_31_x)

0:	learn: 0.0185150	total: 18.6ms	remaining: 2m 47s
500:	learn: 0.0106405	total: 6.39s	remaining: 1m 48s
1000:	learn: 0.0071329	total: 13s	remaining: 1m 43s
1500:	learn: 0.0044514	total: 19.5s	remaining: 1m 37s
2000:	learn: 0.0026863	total: 26s	remaining: 1m 30s
2500:	learn: 0.0016287	total: 32.4s	remaining: 1m 24s
3000:	learn: 0.0010380	total: 39s	remaining: 1m 17s
3500:	learn: 0.0006803	total: 45.7s	remaining: 1m 11s
4000:	learn: 0.0004500	total: 52.5s	remaining: 1m 5s
4500:	learn: 0.0002957	total: 59.4s	remaining: 59.4s
5000:	learn: 0.0001979	total: 1m 6s	remaining: 53.1s
5500:	learn: 0.0001312	total: 1m 13s	remaining: 46.8s
6000:	learn: 0.0000874	total: 1m 20s	remaining: 40.3s
6500:	learn: 0.0000580	total: 1m 27s	remaining: 33.8s
7000:	learn: 0.0000387	total: 1m 35s	remaining: 27.2s
7500:	learn: 0.0000255	total: 1m 42s	remaining: 20.5s
8000:	learn: 0.0000170	total: 1m 49s	remaining: 13.7s
8500:	learn: 0.0000113	total: 1m 57s	remaining: 6.88s
8999:	learn: 0.0000075	total: 2m 4s	rema

In [359]:
catreg2.fit(train_T_31_x, train_T_31_y)
pred_t = catreg2.predict(test_T_31_x)

0:	learn: 0.0089492	total: 5.12ms	remaining: 28.2s
500:	learn: 0.0034069	total: 2.27s	remaining: 22.7s
1000:	learn: 0.0012405	total: 4.58s	remaining: 20.6s
1500:	learn: 0.0004998	total: 6.86s	remaining: 18.3s
2000:	learn: 0.0002111	total: 9.15s	remaining: 16s
2500:	learn: 0.0000834	total: 11.4s	remaining: 13.7s
3000:	learn: 0.0000354	total: 13.8s	remaining: 11.5s
3500:	learn: 0.0000146	total: 16.1s	remaining: 9.18s
4000:	learn: 0.0000062	total: 18.4s	remaining: 6.88s
4500:	learn: 0.0000027	total: 20.7s	remaining: 4.59s
5000:	learn: 0.0000012	total: 23s	remaining: 2.29s
5499:	learn: 0.0000005	total: 25.3s	remaining: 0us


In [360]:
for i in range(0,len(pred_a)):
    if pred_a[i] <= 0.525066667  :
        pred_a[i] = 0
    else : pred_a[i] = 1

In [361]:
for i in range(0,len(pred_t)):
    if pred_t[i] <= 0.525066667  :
        pred_t[i] = 0
    else : pred_t[i] = 1

In [362]:
print(len(pred_a) - sum(pred_a),(len(pred_a) - sum(pred_a))/len(pred_a))

67.0 1.0


In [363]:
print(len(pred_t) - sum(pred_t),(len(pred_t) - sum(pred_t))/len(pred_t))

239.0 1.0


# predict 2

In [382]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)

# {'onehot', 'onehot-dense', 'ordinal'}
from sklearn.preprocessing import KBinsDiscretizer
enc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy="kmeans", random_state=1414)

featurest = train_T_31_x.columns[1:]
enc.fit(train_T_31_x[featurest])
binned_df_train = enc.transform(train_T_31_x[featurest])
binned_df_test = enc.transform(test_T_31_x[featurest])
for i, featurest in enumerate(featurest):
    train_T_31_x[f"{featurest}_binned"] = binned_df_train[:,i]
    test_T_31_x[f"{featurest}_binned"] = binned_df_test[:,i]

enc = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy="kmeans", random_state=1414)
features = trainA_31_x.columns[1:]
enc.fit(trainA_31_x[features])
binned_df_train = enc.transform(trainA_31_x[features])
binned_df_test = enc.transform(testA_31_x[features])
for i, feature in enumerate(features):
    trainA_31_x[f"{feature}_binned"] = binned_df_train[:,i]
    testA_31_x[f"{feature}_binned"] = binned_df_test[:,i]

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 4233) (67, 4233) (343, 1355) (239, 1355)


In [383]:
# binary
trainA_31_w.loc[(trainA_31_w['Y_Class']==0),'Y_Class']=1
train_T_31_w.loc[(train_T_31_w['Y_Class']==0),'Y_Class']=1
train_O_31_w.loc[(train_O_31_w['Y_Class']==0),'Y_Class']=1
traint = pd.concat([train_T_31_x,train_T_31_w],axis = 1)
traina = pd.concat([trainA_31_x,trainA_31_w],axis = 1)

In [384]:
corr_df = traint.corr()
corr_df = corr_df.apply(lambda x: round(x ,5))
s = corr_df.unstack()
성장 = pd.DataFrame(s['Y_Class']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
under_05per_t = 성장[성장[0]<0.001].index
성장 = 성장.sort_values(by=[0],ascending=False).reset_index()
droppc_t = []
for col in traint.columns[1:678] :
    col_corr = traint[[col,col+'_binned']].corr()[col+'_binned'][0]
    if col_corr > 0.70 :        
        origin = 성장[성장['index']==col][0].values[0]
        binned = 성장[성장['index']==col+'_binned'][0].values[0]
        if origin >= binned :
            droppc_t.append(col+'_binned')
        else :
            droppc_t.append(col)
dropt = set(list(under_05per_t)+droppc_t+['LINE'])
print(len(under_05per_t), len(droppc_t))

140 360


In [385]:
corr_df = traina.corr()
corr_df = corr_df.apply(lambda x: round(x ,5))
s = corr_df.unstack()
성장 = pd.DataFrame(s['Y_Class']).sort_values(by=0).fillna(0)
성장[0] = abs(성장[0])
under_05per_a = 성장[성장[0]<0.001].index
성장 = 성장.sort_values(by=[0],ascending=False).reset_index()
droppc_a = []
for col in traina.columns[1:2117] :
    col_corr = traina[[col,col+'_binned']].corr()[col+'_binned'][0]
    if col_corr > 0.70 :    
        origin = 성장[성장['index']==col][0].values[0]
        binned = 성장[성장['index']==col+'_binned'][0].values[0]
        if origin >= binned :
            droppc_a.append(col+'_binned')
        else :
            droppc_a.append(col)
dropa = set(list(under_05per_a)+droppc_a+['LINE'])
print(len(under_05per_a), len(droppc_a))

59 1955


In [386]:
trainA_31_x = trainA_31_x.drop(columns=dropa)
testA_31_x = testA_31_x.drop(columns=dropa)
train_T_31_x = train_T_31_x.drop(columns=dropt)
test_T_31_x = test_T_31_x.drop(columns=dropt)

In [387]:
print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 2223) (67, 2223) (343, 868) (239, 868)


In [388]:
catreg3 = CatBoostRegressor(random_state=1234,
                           depth=5,
                           #eval_metric='MAE',
                           verbose=500,
                           iterations=8000,
                           learning_rate=0.008)

catreg4 = CatBoostRegressor(random_state=1234,
                           depth=5,
                           #eval_metric='MAE',
                           verbose=500,
                           iterations=3000,
                           learning_rate=0.017)

In [389]:
catreg3.fit(trainA_31_x, trainA_31_y)
pred_a2 = catreg3.predict(testA_31_x)

0:	learn: 0.0184945	total: 10.8ms	remaining: 1m 26s
500:	learn: 0.0105652	total: 3.74s	remaining: 55.9s
1000:	learn: 0.0067556	total: 7.42s	remaining: 51.9s
1500:	learn: 0.0041407	total: 11.1s	remaining: 48.2s
2000:	learn: 0.0025259	total: 14.8s	remaining: 44.5s
2500:	learn: 0.0015637	total: 18.6s	remaining: 40.8s
3000:	learn: 0.0009698	total: 22.3s	remaining: 37.1s
3500:	learn: 0.0006195	total: 26.1s	remaining: 33.5s
4000:	learn: 0.0003949	total: 29.9s	remaining: 29.9s
4500:	learn: 0.0002538	total: 33.7s	remaining: 26.2s
5000:	learn: 0.0001666	total: 37.7s	remaining: 22.6s
5500:	learn: 0.0001076	total: 41.6s	remaining: 18.9s
6000:	learn: 0.0000702	total: 45.6s	remaining: 15.2s
6500:	learn: 0.0000454	total: 49.7s	remaining: 11.4s
7000:	learn: 0.0000296	total: 53.7s	remaining: 7.66s
7500:	learn: 0.0000196	total: 57.8s	remaining: 3.85s
7999:	learn: 0.0000129	total: 1m 1s	remaining: 0us


In [390]:
catreg4.fit(train_T_31_x, train_T_31_y)
pred_t2 = catreg4.predict(test_T_31_x)

0:	learn: 0.0089540	total: 6.05ms	remaining: 18.1s
500:	learn: 0.0038989	total: 2.09s	remaining: 10.4s
1000:	learn: 0.0015680	total: 4.24s	remaining: 8.47s
1500:	learn: 0.0006483	total: 6.39s	remaining: 6.38s
2000:	learn: 0.0003117	total: 8.54s	remaining: 4.26s
2500:	learn: 0.0001478	total: 10.7s	remaining: 2.13s
2999:	learn: 0.0000662	total: 12.9s	remaining: 0us


In [391]:
for i in range(0,len(pred_a2)):
    if pred_a2[i] < 0.534950794   :     
        pred_a2[i] = 1
    else : pred_a2[i] = 2

In [392]:
-len(pred_a2) + sum(pred_a2)

0.0

In [393]:
for i in range(0,len(pred_t2)):
    if pred_t2[i] < 0.534950794   :     
        pred_t2[i] = 1
    else : pred_t2[i] = 2

In [394]:
-len(pred_t2) + sum(pred_t2)

0.0

In [377]:
testA_31['Y_2'] = pred_a2
test_T_31['Y_2'] = pred_t2

testA_31['Y_0'] = pred_a
test_T_31['Y_0'] = pred_t

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_0']==0  ), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_2']==2  ), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_0']==0  ), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_2']==2  ), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('corrrlo.csv',index=False)

In [378]:
ddd = pd.read_csv('corrrlo.csv')

In [235]:
kd = pd.read_csv('따로.csv')

In [254]:
ddd['Y_Class'].value_counts()/len(ddd)

1    0.829032
0    0.132258
2    0.038710
Name: Y_Class, dtype: float64

In [379]:
ddd['Y_Class'].value_counts()

0    306
1      4
Name: Y_Class, dtype: int64

In [279]:
kd[kd['Y_Class']==0]

,PRODUCT_ID,Y_Class
3,TEST_003,0
7,TEST_007,0
8,TEST_008,0
13,TEST_013,0
14,TEST_014,0
35,TEST_035,0
36,TEST_036,0
37,TEST_037,0
39,TEST_039,0
41,TEST_041,0


In [380]:
ddd[ddd['Y_Class']==0]

,PRODUCT_ID,Y_Class
0,TEST_000,0
1,TEST_001,0
2,TEST_002,0
3,TEST_003,0
4,TEST_004,0
...,...,...
305,TEST_305,0
306,TEST_306,0
307,TEST_307,0
308,TEST_308,0


In [381]:
ddd[ddd['Y_Class']==2]

,PRODUCT_ID,Y_Class


In [205]:
kd[kd['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
109,TEST_109,2
115,TEST_115,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
169,TEST_169,2
177,TEST_177,2
183,TEST_183,2
